# Partially Observed Data

### Pseudo-code

<pre>
<i><b>Expectation-Maximization</b></i> (
    &theta;<sup>(0)</sup> // Initial parameters
    D   // Data
  )
1    <b>for</b> each iteration t
2        M<sub>t</sub> &larr; E-step(&theta;<sup>(t)</sup>, D)
3        &theta;<sup>(t+1)</sup> &larr; M-step(M<sub>t</sub>)
4    <b>return</b> &theta;<sup>(T)</sup>

<i><b>E-step</b></i> (
    &theta;<sup>(t)</sup> // Current parameters
    D   // Data
  )
1    M<sub>t</sub> &larr; 0
2    <b>for</b> each data point d
3        <b>for</b> each node X
4            <b>for</b> each combination X=x, Pa(X)=y
5                M<sub>t</sub>[x, y] &larr; M<sub>t</sub>[x, y] + p(x, y|d, &theta;<sup>(t)</sup>)
6    <b>return</b> M<sub>t</sub>
    

<i><b>M-step</b></i> (
    M<sub>t</sub>  // Expected sufficient statistics
  )
1    <b>for</b> each node X
2        <b>for</b> each combination X=x, Pa(X)=y
3            &theta;<sup>(t+1)</sup>[x|y] &larr; M<sub>t</sub>[x, y] / M<sub>t</sub>[p]
4    <b>return</b> &#x03B8;<sup>(t+1)</sup>
</pre>

In [ ]:
import numpy as np

from em import expectation_maximization, generate_data, print_tables, print_marginals

In [ ]:
def e_step(qx, qy, qz, xs, ys, zs):
    Mx = np.zeros(2)
    My = np.zeros(2)
    Mz = np.zeros((2, 2, 2))  # Remember index order: Mz[x, y, z]
    for x, y, z in zip(xs, ys, zs):
        """
        To do:  p(X|x, y, z) and add to Mx
                p(Y|x, y, z) and add to My
                p(Z, X, Y|x, y, z) and add to Mz.
            Remember to normalize p(.), i.e. each should sum to 1. 
                For example, if x, y and z are not None, we should have p(X=x) = 1, p(Y=y) = 1, p(Z=z, X=x, Y=y) = 1. 
        Naive solution (~45 lines of code): 
            x and y are None? ...
            x is None: ... (NB: p(X|Y=y, Z=z) = p(X, Y=y, Z=z) / p(Y=y, Z=z) != p(X))
            y is None: ...
            x and y are known: p(...) = 1
        Pythonic solution (<10 lines of code):
            Q = np.zeros((2, 2)) # Q(x, y) = p(Z=z, X=x, Y=y)
            # Q <- p(...)
            Mx += ...
            My += ...
            Mz[:, :, z] += ...
        """
    return Mx, My, Mz

In [ ]:
def m_step(Mx, My, Mz):
    """
    Convert from sufficient statistics to parameters. What elemets should sum to one?
    """
    return qx, qy, qz

## Run

In [ ]:
np.random.seed(1337)
px = np.array([0.6, 0.4])
py = np.array([0.3, 0.7])
pz = np.array([[[0.2, 0.8], [0.7, 0.3]], [[0.9, 0.1], [0.1, 0.9]]])  # p(z|x, y) = pz[x, y, z]
n_data = 500
x, y, z = generate_data(px, py, pz, n_data)
n_iter = 10
qx, qy, qz = expectation_maximization(x, y, z, e_step, m_step, n_iter)

In [ ]:
print("Learnt parameters")
print("-----------------")
print_tables(qx, qy, qz)
print()
print("True parameters")
print("---------------")
print_tables(px, py, pz)

In [ ]:
print("Learnt marginals")
print("----------------")
print_marginals(qx, qy, qz)
print()
print("True marginals")
print("--------------")
print_marginals(px, py, pz)